In [4]:
import requests
import json
import requests
import pandas as pd
from plotly import graph_objects as go
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
%run trips_duration.ipynb

object


/var/folders/mv/1xmk4xds7d548s_3ktkk13600000gn/T/ipykernel_15661/1627526132.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
def merging_data(final_1plus_df):

    tourist_num = final_1plus_df
    tourist_num_copy = final_1plus_df.copy()

    min_trips = tourist_num['NumTrips'].min()
    max_trips = tourist_num['NumTrips'].max()

    tourist_num['Tourism St.'] = (tourist_num['NumTrips'] - min_trips) / (max_trips - min_trips)

    tourist_num = pd.DataFrame(final_1plus_df, columns = ['Country', 'Year', 'Tourism St.'])
    tourist_num.sort_values(by = ["Year"])

    unfiltered_predicted_tourist =pd.DataFrame(tourist_num_copy, columns = ["Country", "Year", "NumTrips"])

    return tourist_num, unfiltered_predicted_tourist


In [7]:
datas = merging_data(final_1plus_df)
data = datas[0]
data_not_st = datas[1]
data

,Country,Year,Tourism St.
0,Albania,2017,0.014321
1,Albania,2019,0.018106
2,Albania,2020,0.011349
3,Austria,2012,0.067236
4,Austria,2013,0.068143
...,...,...,...
345,Switzerland,2018,0.076381
346,Switzerland,2019,0.069107
347,Switzerland,2020,0.059142
348,Switzerland,2021,0.061503


In [8]:
def get_lag_columns(country_df, lag = 3):

    country_df = country_df.sort_values('Year').reset_index(drop = True)
    country_df['Covid'] = country_df['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
    country_df['Bias'] = 1
    min_year = country_df['Year'].min()
    
    index_country = country_df.index[country_df['Year'].astype(int) == min_year + 3]
    index = index_country[0]

    y_vector = []
    x_matrix = []
    x_list = []
    for i in range(index, len(country_df)): 
        if i - lag < 0:
            continue
        bias = 1 
        country = country_df.iloc[i]["Country"]
        lag_values = [country_df.iloc[i - j]['Tourism St.'] for j in range(1, lag + 1)]
        covid = country_df.iloc[i]['Covid']
        year = country_df.iloc[i]['Year']
        
        x_list = [bias, country, year, covid] + lag_values
        x_matrix.append(x_list)
        y_vector.append(country_df.iloc[i]["Tourism St."])

    x_matrix = pd.DataFrame(x_matrix)
    x_matrix.rename(columns={0: "Bias", 1: "Country", 2:"Year", 3:"Covid", 4:"lag1", 5:"lag2", 6:"lag3"}, inplace=True)


    return x_matrix, y_vector

In [9]:
data[data['Country']=='Albania']

,Country,Year,Tourism St.
0,Albania,2017,0.014321
1,Albania,2019,0.018106
2,Albania,2020,0.011349


In [10]:
X, y = get_lag_columns(data[data['Country'] == 'Albania'], lag=3)
print(X)
print(y)

Empty DataFrame
Columns: []
Index: []
[]


In [15]:
def fit_regression(X, y):
    Xt = X.T
    XtX = np.dot(Xt, X)
    XtX_inv = np.linalg.inv(XtX)
    Xt_y = np.dot(Xt, y)
    beta = np.dot(XtX_inv, Xt_y)

    return beta

In [16]:
def predict_next_year(beta, X):
    return np.dot(beta, X)

In [94]:
def get_country_years_pred(data, country, lag = 3):

    country_df = data[data['Country'] == country]
    country_df, y = get_lag_columns(country_df, lag = lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias', 'Covid'] + lag_cols
    
    X = country_df[feature_cols].values

    beta_tourism = fit_regression(X, y)

    predictions = []
    for index, row in country_df.iterrows():
        year = row['Year']
        X_year = np.array([row[col] for col in feature_cols])
        pred_tourism = predict_next_year(beta_tourism, X_year)

        predictions.append({
            'Country': country,
            'Year': year,
            'Actual_Tourism': y[index],
            'Predicted_Tourism': pred_tourism

        })

    all_years_results = pd.DataFrame(predictions)

    return all_years_results

In [95]:
prev_year_result = get_country_years_pred(data, "Slovakia")
prev_year_result

,Country,Year,Actual_Tourism,Predicted_Tourism
0,Slovakia,2015,0.025319,0.029939
1,Slovakia,2016,0.030385,0.031425
2,Slovakia,2017,0.034212,0.035546
3,Slovakia,2018,0.039610,0.041058
4,Slovakia,2019,0.044606,0.047150
5,Slovakia,2020,0.019206,0.024017
6,Slovakia,2021,0.023941,0.019130
7,Slovakia,2022,0.041598,0.040566
8,Slovakia,2023,0.048077,0.038123


In [96]:
r2_score(prev_year_result['Actual_Tourism'], prev_year_result['Predicted_Tourism'])

0.7782708312971978

In [101]:
def get_country_prediction(data, country, lag= 3):
    country_df = data[data['Country'] == country].copy()
    country_df, y = get_lag_columns(country_df, lag=lag)

    lag_cols = [f'lag{i}' for i in range(1, lag + 1)]
    feature_cols = ['Bias', 'Covid'] + lag_cols

    X = country_df[feature_cols].values

    beta_tourism = fit_regression(X, y)
    #print(country, beta_tourism)
    last_year = country_df['Year'].max()
    #print(last_year)

    last_row = country_df[country_df['Year'] == last_year].iloc[0]
    lag_values = [last_row[f'lag{i}'] for i in range(1, lag + 1)]

    tmp_x = last_row[feature_cols].to_numpy()
    x_old = np.hstack([tmp_x[:2], y[-1], tmp_x[2:4]])
    
    year_list = [last_year + 1]
    country_list = [country]
    pred_list = [predict_next_year(beta_tourism, x_old)]

    for i in range(1, lag):
        year = last_year + i + 1
        
        X_new = np.hstack([x_old[:2], pred_list[i-1], x_old[2:4]])
        pred_tourism = predict_next_year(beta_tourism, X_new)

        year_list.append(year)
        country_list.append(country)
        pred_list.append(pred_tourism)

        x_old = X_new

    predictions = {'Year': year_list,
                   'Country': country_list,
                   'Predicted_Tourism': pred_list}

    return pd.DataFrame(predictions)


In [102]:
EU_countries_list = [
    'Austria', "Belgium", "Bulgaria", "Croatia", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
    "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
    "Spain", "Sweden"]

np.set_printoptions(suppress=True)
for country in EU_countries_list:
    pred = get_country_prediction(data, country, lag = 3)
#print(pred)

In [126]:
rows = []
for country in EU_countries_list:
    result = get_country_prediction(data, country, lag=3)
    result.sort_values(by=['Year'])
    pred_2024 = result[result["Year"] == 2024]["Predicted_Tourism"].values
    pred_2025 = result[result["Year"] == 2025]["Predicted_Tourism"].values
    pred_2026 = result[result["Year"] == 2026]["Predicted_Tourism"].values

    rows.append({'Countries': country, 
                 2024: pred_2024, 
                 2025: pred_2025, 
                 2026: pred_2026})
    
curse = pd.DataFrame(rows)
curse



,Countries,2024,2025,2026
0,Austria,[0.0642708472483588],[-0.005506772971460943],[0.024317792363258414]
1,Belgium,[0.1179998123542246],[0.17381889623601515],[0.2318432367818546]
2,Bulgaria,[0.015586399983443362],[0.012531172468649704],[0.01214093264671625]
3,Croatia,[0.020529424378194733],[0.020877852585991084],[0.017686374146107874]
4,Czechia,[0.12011361128834523],[0.1253039811750993],[0.12114811160789163]
5,Denmark,[0.07926327890117552],[0.08289596746143145],[0.08447657355782935]
6,Estonia,[0.01581564389834277],[0.01896414512228525],[0.018653181221900142]
7,Finland,[0.1276258786812182],[0.11949939175794819],[0.12704085608454876]
8,France,[0.6385783465097844],[0.6229117211392727],[0.6933253422701249]
9,Germany,[0.7663962521321276],[0.7035460308583598],[0.7137472386902484]


In [114]:
result.sort_values(by=['Year'])
results_t = result.T
print(results_t)

                          0         1         2
Year                   2023      2024      2025
Country              Sweden    Sweden    Sweden
Predicted_Tourism  0.282407  0.305952  0.458802


In [81]:
def visualization(data, st_data): 
    min_trips = data['NumTrips'].min()
    max_trips = data['NumTrips'].max()

    st_data["Predicted_Tourism_normal"] = st_data["Predicted_Tourism"] * (max_trips - min_trips) + min_trips

    
    country_name = st_data["Country"].iloc[0]
    matching_data = data[data["Country"] == country_name]

    merged = pd.concat([st_data, matching_data], ignore_index=True)
    merged = merged.sort_values(by="Year").reset_index(drop=True)

    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])

    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=merged['Year'], 
        y=merged['Tourism_combined'], 
        name='Tourist Numbers',
        line=dict(color='royalblue', width=4),
    ))

    fig.add_vline(
        x=2023,  
        line=dict(color='gray', dash='dot'),
        annotation_text='Prediction',
        annotation_position='top right'
)
    fig.update_layout(
        title = "Predicted Tourism Over the Years", 
        xaxis_title = 'Years',
        yaxis_title = 'Tourism Numbers (Millions)')

    fig.show()


In [ ]:
visualization(data_not_st, result)

In [ ]:
pred_df_all = []
y_all = []
EU_countries_list = [
    'Austria', "Belgium", "Bulgaria", "Croatia", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
    "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
    "Spain", "Sweden"]
#EU_countries_list = ['Slovakia']

# loop through all the countries
for country in EU_countries_list:
    country_df = data[data['Country'] == country].copy()
# for each country
# drop the last three years from data (call this subset_data or somethihg)
    max_year = country_df["Year"].max()
    if max_year < 2023:
        subset_data = country_df[country_df['Year'] < max_year - 1]
    else:
        subset_data = country_df[country_df['Year'] < max_year - 2]
    
    temp_df = get_country_prediction(subset_data, country, lag=3)
    #print(temp_df)

# concantenate temp_df to the bottom of pred_df
    pred_df_all.append(temp_df)

    #pred_df_all
# when you are done, compare the pred_df predictions to a vector y_true
# where y_true are the 2021-2023 y values for the countries
    if max_year < 2023:
        y_true = country_df[country_df['Year'] > max_year - 2]
    else:
        y_true = country_df[country_df['Year'] > max_year - 3]
    #print(y_true)
    y_all.append(y_true)

pred_df_true = pd.concat(pred_df_all, ignore_index=True)
y_fin = pd.concat(y_all, ignore_index=True)

results = pd.merge(pred_df_true, y_fin, on = ['Country', 'Year'])

results

results[results["Country"] == "Spain"]


,Year,Country,Predicted_Tourism,Tourism St.
72,2021,Spain,0.126263,0.423896
73,2022,Spain,-0.031529,0.523135
74,2023,Spain,-0.045168,0.552028


In [ ]:
def evaluate_predictions(pred_df):
    r2_tourism = np.corrcoef(pred_df['Tourism St.'], pred_df['Predicted_Tourism'])[0,1]**2
    mse_tourism = mean_squared_error(pred_df['Tourism St.'], pred_df['Predicted_Tourism'])

    evaluations = {
        'Tourism': {'R2': r2_tourism, 'MSE': mse_tourism}
    }

    return evaluations

In [ ]:
evaluate_predictions(results)

{'Tourism': {'R2': 0.0600039743459821, 'MSE': 0.2568171722405778}}